In [ ]:
!pip install numpy
!pip install matplotlib
!pip install wquantiles
!pip install scipy
!pip install sklearn

In [ ]:
from functools import partial
from util.aggregators import mean, median, trimmed_mean, gamma_mean, geometric_median
#from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD, IncrementalPCA

import numpy as np
import matplotlib.pyplot as plt

def generate(p, m, gen_par):
  """
  shape (m, p), p:#parameters, m:#machines
  """
  if gen_par["distribution"]=="multi_norm":
    return gen_par["generator"](gen_par["mu"], gen_par["sigma"], m)
  elif gen_par["distribution"]=="t_dist":
    return gen_par["generator"](gen_par["location"], gen_par["shape"], gen_par["df"], m)

#another usage
#not sure which one is more convenient for our simulation
#def generate(p, m, generator, *par):
#  """
#  shape (m, p), p:#parameters, m:#machines
#  """
#  if generator=="multi_norm":
#    return np.random.multivariate_normal(*par, size = m)
#data = generate(p=100, m=20, generator="multi_norm", mu, sigma)

"""
return data should be v1,v2,...
v1,v2,... are np.array
"""

def bias(y_pred, y_true):
    """
    y_pred: (max_ite, p)
    y_true: (p,)
    """
    return np.square(np.subtract(np.mean(y_pred, axis = 0),y_true)).mean()
bias.__name__="bias"

def mse(y_pred, y_true):
    """
    y_pred: (max_ite, p)
    y_true: (p,)
    """
    return np.mean([np.square(np.subtract(y_hat,y_true))  for y_hat in y_pred], axis = 0).mean()
mse.__name__="mse"

def variance(y_pred, y_true):
    """
    y_pred: (max_ite, p)
    y_true: (p,)
    """
    return np.mean([np.square( np.subtract( y_hat, np.mean(y_pred, axis = 0) ) )  for y_hat in y_pred], axis = 0).mean()
variance.__name__="variance"

def compute_result(estimate, truth, metric):
    """
    metric = [bias, mse, ...]
    """
    result = []
    for fn in metric:
        result.append(fn(estimate,truth))
    return np.array(result)

In [ ]:
"""
p: dimension
m: machines
b: byz_machines
"""

def Gaussian_gen(mu, sigma):
    return {"distribution": "multi_norm",
          "generator": np.random.multivariate_normal,
          "mu": mu,
          "sigma": sigma 
          } 
        
def t_gen(mu, sigma, df):
    def multivariate_t_rvs(mu, S, df ,size):
        '''generate random variables of multivariate t distribution
        Parameters
        ----------
        mu : array_like
            mean of random variable, length determines dimension of random variable
        S : array_like
            square array of covariance  matrix
        df : int or float
            degrees of freedom
        size : int
            number of observations, return random array will be (size, len(mu))
        Returns
        -------
        rvs : ndarray, (size, len(mu))
            each row is an independent draw of a multivariate t distributed
            random variable
        '''
        mu = np.asarray(mu)
        d = len(mu)
        if df == np.inf:
            x = 1.
        else:
            x = np.random.chisquare(df, size)/df
        z = np.random.multivariate_normal(np.zeros(d),S,(size,))
        return mu + z/np.sqrt(x)[:,None]
    return {"distribution": "t_dist",
          "generator": multivariate_t_rvs,
          "location": mu,
          "shape": sigma,
          "df":df
          }

In [ ]:
"""
Scenario 1
multiple experiments 

fix m=200 alpha=0 
data from T distribution(df=3)
when gamma=0.01
get station and low bias/p when p goes to 1000
bias/p about 0.01

data from multi-normal distribution
when gamma=0.01
get station and low bias/p when p goes to 1000
bias/p about 0.005
"""
#set seed
seed = 2021


m = 200 #Total machines
alpha = 0
b = int(m*alpha) #Byzantine machines
simu_iter = 100 #number of experiments
#gamma = 0.005
beta = 0.1
# p: dimension 
p_range = np.arange(20,1000,20)
t_df = 5 # degree of freedom in t distribution

metrics = [bias, mse, variance] #plot as below

"""
for i, metric in enumerate(metrics):
  plot results[dist][method][i_metrics,:] 
  ylab metric
""" 

dist_list = ["Gaussian", "T_dist"]  #for convenience if one day we want to add more generate dist
#declare a dictionary to map the generators to functions
dist2fun = {"Gaussian":Gaussian_gen, 
            "T_dist":partial(t_gen, df = t_df)
            }

#for convenient if one day we want to add more aggregator
method_list = ["mean","median","gamma","geo","trim"]

#declare a dictionary, it is very convenient to add more object inside
results=dict()

#iterate all generators
for dist in dist_list:
  #to restore results
  temp_result = dict()
  for method in method_list:
    temp_result[method] = np.zeros((len(metrics),len(p_range)))

  for p in p_range:
    mu = [0]*p
    mu_byz = [100]*p
    index = int((p / 20) - 1)
    gamma = 2/p
    #declare a dictionary to map the methods to functions
    method2fun = {"mean":partial(mean, weights = None),
                  "median":partial(median, weights = None),
                  "gamma":partial(gamma_mean, weights = None, compute = 'simple', gamma = gamma),
                  "geo":partial(geometric_median, weights = None),
                  "trim":partial(trimmed_mean, weights = None, beta = beta)
                  }
    
    gen_par = dist2fun[dist](mu, sigma=np.eye(p))
    byz_par = dist2fun[dist](mu_byz, sigma=np.eye(p))

    #to restore the estimation
    temp = dict()
    for method in method_list:
      temp[method]=[]
    #iterate many times
    for t in range(simu_iter):
      if int(p/20)==p/20:
        print("Iterate: {} for dimension: {} with data generate from {}".format(t,p,dist))
      np.random.seed(seed + 100*t + p)
      input = np.concatenate((generate(p, m-b, gen_par), generate(p, b, byz_par)), axis = 0) if b>0 else generate(p, m, gen_par)
      for method in method_list:
        temp[ method ].append( method2fun[ method ]( input ) )
      #end repeating simulation for-loop
    for method in method_list:
      temp_result[method][:,index] = compute_result(temp[ method ], mu, metrics)
    #end p_range for-loop
  results [dist] = temp_result
  #end dist_list for-loop

In [ ]:
#plot the comparison of aggregators
fig, axs = plt.subplots(3, 2, figsize=(20, 10), sharex=True, sharey='row',  gridspec_kw={'hspace': 0.2, 'wspace': 0.04})
(ax1, ax2),  (ax3, ax4), (ax5, ax6)= axs

ax1.set_title("Gaussian")
ax2.set_title('t-distribution(df=5)')
right = [ax2, ax4, ax6]

for i, metric in enumerate(metrics):
  for j, dist in enumerate(dist_list):
    ax = axs[i][j]
    for method in method_list:
      r = results[dist][method][i,:]
      ax.plot(p_range, r, label = method) 
      ax.set_xlabel('p')
      ax.set_xticks(p_range[np.arange(4,len(p_range),5)])
    ax.set_ylabel(metric.__name__)
for ax in right:
  #method_list is ["mean","median","gamma","geo","trim"] need to use the same order
  ax.legend(labels=['mean', 'median', '\u03B3-mean with gamma=2/p', 'GeoMed', 'Trimmed-mean with beta = {}'.format(beta)],
            loc='upper right',bbox_to_anchor=(1.4,1.0))
plt.show()
#fig.savefig('attackers_0%.pdf', format='pdf', bbox_inches='tight')

In [ ]:
"""
Scenario 2
multiple experiments 

fix m=200 p=1000 
data from multi-normal distribution
when gamma=0.1
get station and low bias/p when alpha goes to 0.5
bias/p about 0.8 (senario 1: 0.01)
"""
seed = 2021


m = 200 #Total machines
simu_iter = 100 #number of experiments
#gamma = 0.005
beta = 0.1
# p: dimension 
p = 1000
alpha_range = np.arange(0.05,0.5,0.05)

metrics = [bias, mse, variance] #plot as below

"""
for i, metric in enumerate(metrics):
  plot results[dist][method][i_metrics,:] 
  ylab metric
""" 

dist_list = ["Gaussian", "T_dist"]  #for convenience if one day we want to add more generate dist
#declare a dictionary to map the generators to functions
dist2fun = {"Gaussian":Gaussian_gen, 
            "T_dist":partial(t_gen, df = 5)
            }

#for convenient if one day we want to add more aggregator
method_list = ["mean","median","gamma","geo","trim"]

#declare a dictionary, it is very convenient to add more object inside
results=dict()

#iterate all generators
for dist in dist_list:
  #to restore results
  temp_result = dict()
  for method in method_list:
    temp_result[method] = np.zeros((len(metrics),len(alpha_range)))

  for alpha in alpha_range:
    b = int(m*alpha)
    mu = [0]*p
    mu_byz = [100]*p
    index = int(20*alpha - 1)
    gamma = 0.5/p
    #declare a dictionary to map the methods to functions
    method2fun = {"mean":partial(mean, weights = None),
                  "median":partial(median, weights = None),
                  "gamma":partial(gamma_mean, weights = None,compute = 'simple', gamma = gamma),
                  "geo":partial(geometric_median, weights = None),
                  "trim":partial(trimmed_mean, weights = None, beta = beta)
                  }
    
    gen_par = dist2fun[dist](mu, sigma=np.eye(p))
    byz_par = dist2fun[dist](mu_byz, sigma=np.eye(p))

    #to restore the estimation
    temp = dict()
    for method in method_list:
      temp[method]=[]
    #iterate many times
    for t in range(simu_iter):
      np.random.seed(seed + 100*t + int(1000*alpha))
      input = np.concatenate((generate(p, m-b, gen_par), generate(p, b, byz_par)), axis = 0) if b>0 else generate(p, m, gen_par)
      for method in method_list:
        temp[ method ].append( method2fun[ method ]( input ) )
      #end repeating simulation for-loop
    for method in method_list:
      temp_result[method][:,index] = compute_result(temp[ method ], mu, metrics)
    #end alpha_range for-loop
  results [dist] = temp_result
  #end dist_list for-loop     

In [ ]:
#plot the comparison of aggregators
fig, axs = plt.subplots(3, 2, figsize=(20, 10), sharex=True, sharey='row',  gridspec_kw={'hspace': 0.2, 'wspace': 0.04})
(ax1, ax2),  (ax3, ax4), (ax5, ax6)= axs
ax1.set_title("Gaussian")
ax2.set_title('t-distribution(df=5)')
right = [ax2, ax4, ax6]

for i, metric in enumerate(metrics):
  for j, dist in enumerate(dist_list):
    ax = axs[i][j]
    for method in method_list:
      r = results[dist][method][i,:]      
      ax.plot(alpha_range, r, label = method) 
      ax.set_xlabel('\u03B1') #alpha
      #ax.set_xticks(alpha_range[np.arange(4,len(alpha_range),5)])
    ax.set_ylabel(metric.__name__)
for ax in right:
  #method_list is ["mean","median","gamma","geo","trim"] need to use the same order
  ax.legend(labels=['mean', 'median', '\u03B3-mean with \u03B3 = 2/p', 'GeoMed', 'Trimmed-mean with \u03B2 = {}'.format(beta)],
            loc='upper right',bbox_to_anchor=(1.4,1.0))
  
plt.suptitle('Testing different aggregatros across \u03B1 ',fontsize=25)
plt.show()

$(1+\gamma)^{\frac{p+2}{2}}e^{\frac{-\gamma}{2}(x_0-\mu)^\mathrm{T}\mathbf{\Sigma^{-1}}(x_0-\mu)}(x_0-\mu) $

Althogh theory indicates that $\gamma$-divergence reduces to KL-divergence as $\gamma\to 0$ , it seems like there is also a relation bewteen $p$ and $\gamma$, assume to be $\gamma=g(p)$, s.t. $\|\mu- \hat\mu\|<c$  
 
 Maybe $p\times\gamma<0.1$.  

In [ ]:
"""    
experiment on gamma with p
scenario 3
"""
    
seed = 2021


m = 200 #Total machines
alpha = 0.1 #Byzantine fraction
simu_iter = 100 #number of experiments
beta = 0.1
# p: dimension 
p_range = np.arange(20,1000,20)


metrics = [bias, mse, variance] #plot as below

"""
for i, metric in enumerate(metrics):
  plot results[dist][method][i_metrics,:] 
  ylab metric
""" 

dist_list = ["Gaussian", "T_dist"]  #for convenience if one day we want to add more generate dist
#declare a dictionary to map the generators to functions
dist2fun = {"Gaussian":Gaussian_gen, 
            "T_dist":partial(t_gen, df = 5)
            }

#for convenient if one day we want to add more aggregator
method_list = ["gamma_0.5/p","gamma_1/p","gamma_2/p","gamma_3/p","gamma_4/p"]

#declare a dictionary, it is very convenient to add more object inside
results={"Gaussian":None }

#iterate all generators
for dist in dist_list:
  #to restore results
  temp_result = dict()
  for method in method_list:
    temp_result[method] = np.zeros((len(metrics),len(p_range)))

  for p in p_range:
    b = int(m*alpha)
    mu = [0]*p
    mu_byz = [100]*p
    index = int((p / 20) - 1)
    gamma = [0.5/p, 1/p, 2/p, 3/p, 4/p]
    #declare a dictionary to map the methods to functions
    method2fun = {"gamma_0.5/p":partial(gamma_mean, weights = None, gamma = gamma[0]),
                  "gamma_1/p":partial(gamma_mean, weights = None, compute = 'simple',  gamma = gamma[1]),
                  "gamma_2/p":partial(gamma_mean, weights = None, compute = 'simple', gamma = gamma[2]),
                  "gamma_3/p":partial(gamma_mean, weights = None, compute = 'simple', gamma = gamma[3]),
                  "gamma_4/p":partial(gamma_mean, weights = None, compute = 'simple', gamma = gamma[4])
                  }
    
    gen_par = dist2fun[dist](mu, sigma=np.eye(p))
    byz_par = dist2fun[dist](mu_byz, sigma=np.eye(p))

    #to restore the estimation
    temp = dict()
    for method in method_list:
      temp[method]=[]
    #iterate many times
    for t in range(simu_iter):
      np.random.seed(seed + 100*t + p)
      input = np.concatenate((generate(p, m-b, gen_par), generate(p, b, byz_par)), axis = 0) if b>0 else generate(p, m, gen_par)
      for method in method_list:
        temp[ method ].append( method2fun[ method ]( input ) )
      #end repeating simulation for-loop
    for method in method_list:
      temp_result[method][:,index] = compute_result(temp[ method ], mu, metrics)
    #end alpha_range for-loop
  results [dist] = temp_result
  #end dist_list for-loop

In [ ]:
#plot comparison of different gammas
fig, axs = plt.subplots(3, 2, figsize=(20, 10), sharex=True, sharey='row',  gridspec_kw={'hspace': 0.2, 'wspace': 0.04})

(ax1, ax2),  (ax3, ax4), (ax5, ax6)= axs
ax1.set_title("Gaussian")
ax2.set_title('t-distribution(df=5)')
right = [ax2, ax4, ax6]

for i, metric in enumerate(metrics):
  for j, dist in enumerate(dist_list):
    ax = axs[i][j]
    for method in method_list:
      r = results[dist][method][i,:]      
      ax.plot(p_range, r, label = method)
      ax.set_xlabel('p')
      ax.set_xticks(p_range[np.arange(4,len(p_range),5)])
    ax.set_ylabel(metric.__name__)

for ax in right:
  #method_list is ["mean","median","gamma","geo","trim"] need to use the same order
  ax.legend(labels=['\u03B3 = 0.5/p', '\u03B3 = 1/p', '\u03B3 = 2/p','\u03B3 = 3/p', '\u03B3 = 4/p']
            ,loc='upper right',bbox_to_anchor=(1.2,1.0))
plt.suptitle('Testing different \u03B3 across dimension p, \u03B1 = {}'.format(alpha),fontsize=25)
plt.show()

In [ ]:
"""
fix m=200 alpha=0.1
data from T distribution(df=3)

test gamma_mean 2D and 1D

Some bugs

"""
#set seed
seed = 2021


m = 200 #Total machines
alpha = 0.1
b = int(m*alpha) #Byzantine machines
simu_iter = 100 #number of experiments

# p: dimension 
p_range = np.arange(20,1000,20)

metrics = [bias, mse, variance] #plot as below
"""
for i, metric in enumerate(metrics):
  plot results[dist][method][i_metrics,:] 
  ylab metric
""" 

dist_list = ["Gaussian", "T_dist"]  #for convenience if one day we want to add more generate dist
#declare a dictionary to map the generators to functions
dist2fun = {"Gaussian":Gaussian_gen, 
            "T_dist":partial(t_gen, df = 5)
            }

#for convenient if one day we want to add more aggregator
method_list = ["gamma_1D","gamma_2D"]

#declare a dictionary, it is very convenient to add more object inside
results=dict()

#iterate all generators
for dist in dist_list:
  #to restore results
  temp_result = {"gamma_1D":None,
                 "gamma_2D":None,}
  for method in method_list:
    temp_result[method] = np.zeros((len(metrics),len(p_range)))

  for p in p_range:
    mu = [0]*p
    mu_byz = [100]*p
    index = int((p / 20) - 1)
    gamma = 2/p
    #declare a dictionary to map the methods to functions
    method2fun = {"gamma_1D":partial(gamma_mean, weights = None, compute = "1D", gamma = gamma),
                  "gamma_2D":partial(gamma_mean, weights = None, compute = "2D", gamma = gamma, dim_red = True)
                  }
    gen_par = dist2fun[dist](mu, sigma=np.eye(p))
    byz_par = dist2fun[dist](mu_byz, sigma=np.eye(p))

    #to restore the estimation
    temp = {"gamma_1D":None}
    for method in method_list:
      temp[method]=[]
    #iterate many times
    for t in range(simu_iter):
      print("Iterate: {}".format(t))
      np.random.seed(seed + 100*t + p)
      input = np.concatenate((generate(p, m-b, gen_par), generate(p, b, byz_par)), axis = 0) if b>0 else generate(p, m, gen_par)
      for method in method_list:
        temp[ method ].append( method2fun[ method ]( input ) )
      #end repeating simulation for-loop
    for method in method_list:
      temp_result[method][:,index] = compute_result(temp[ method ], mu, metrics)
    #end p_range for-loop
  results [dist] = temp_result
  #end dist_list for-loop

In [ ]:
#plot the comparison of aggregators
fig, axs = plt.subplots(3, 2, figsize=(20, 10), sharex=True, sharey='row',  gridspec_kw={'hspace': 0.2, 'wspace': 0.04})
(ax1, ax2),  (ax3, ax4), (ax5, ax6)= axs

ax1.set_title("Gaussian")
ax2.set_title('T distribution(df=10)')
right = [ax2, ax4, ax6]

for i, metric in enumerate(metrics):
  for j, dist in enumerate(dist_list):
    ax = axs[i][j]
    for method in method_list:
      r = results[dist][method][i,:]
      ax.plot(p_range, r, label = method) 
      ax.set_xlabel('p')
      ax.set_xticks(p_range[np.arange(4,len(p_range),5)])
    ax.set_ylabel(metric.__name__)
for ax in right:
  #method_list is ["mean","median","gamma","geo","trim"] need to use the same order
  ax.legend(labels=['mean', 'median', 'gamma mean with gamma=2/p', 'geometric median', 'trim mean with beta = {}'.format(beta)],
            loc='upper right',bbox_to_anchor=(1.4,1.0))
plt.show()